# Análisis exploratorio de datos (EDA)
### Autores: Ricardo, Eusebio, Marcos, Adrián, Javier Gamero Muñoz 

# Índice 
* [Introducción al problema](#introducción-al-problema)
* [Recodificación de variables](#recodificación-de-variables)
  * [Variable ```Cabin```](#recodificación-de-la-variable-cabin)
  * [Creación de la variable ```Alone```](#creación-de-la-variable-alone)
* [Valores perdidos / *Missing values*](#valores-perdidos-o-missing-values)
  * [Missing values categóticos](#missing-values-categóricos)
  * [Missing values numéricos](#missing-values-numéricos)

# Introducción al problema
El problema que se presenta tiene lugar en el año 2912, donde la nave espacial 
*Titanic* colisionó con una anomalía espacial. Como resultado de este accidente, 
algunos de los pasajeros fueron transportados a otra dimensión, y el objetivo es 
clasificar (a partir de aquellos que sabemos que han sido transportados,
**train**) si un menor grupo de personas (**test**) han sido transportados o 
no.  
Para ello se hará uso de cinco algoritmos de clasificación: 
* *k nearest neighbors (kNN)* 
* Árboles de clasificación
* *Supported vector machine (SVM)*
* Regresión logística
* Naïve Bayes

Los datos recogidos están separados en dos datasets, ```train.csv``` y 
```test.csv```, teniendo el primero de ellos los casos donde sí se conoce el 
destino de las personas (si han sido transportadas o no), mientras que en el 
segundo de ellos no. 
Comenzamos estudiando las variables del dataset ```train.csv```:

In [ ]:
library(tidyverse)
library(Amelia)

In [ ]:
df = read.csv("data/train.csv", header = TRUE)
head(df)
str(df)

Es posible observar que se tienen 8693 observaciones y 14 variables, de las 
cuales 8 están formadas por strings y el resto son numéricas.   
De la página de 
[kaggle](https://www.kaggle.com/competitions/spaceship-titanic/data)
con los datos se obtiene la siguiente información de los atributos:
* ```PassengerId```: identifiación para cada pasajero en el formato *gggg_pp* 
  donde *gggg* indica su grupo de viaje y *pp* su número en él. Personas en un 
  mismo grupo suelen ser familias, aunque no siempre.
* ```Homeplanet```: planeta donde embarca el pasajero.
* ```CryoSleep```: indica si el pasajero fue puesto en animación suspendida 
  durante el viaje.
* ```Cabin```: número de cabina donde el pasajero viajaba en el formato 
  *deck/num/side* donde *side* puede ser *P* para *babor* o *S* para *estribor*.
* ```Destination```: planeta donde el pasajero desembarca.
* ```Age```: edad del pasajero.
* ```VIP```: indica si el pasajero tenía un servicio VIP durante el viaje.
* ```RoomService```, ```FoodCourt```, ```ShoppingMall```, ```Spa```, 
  ```VRDeck```: cantidad de dinero empleado en los distintos servicios de la nave.
* ```Name```: nombre y primer apellido del pasajero.
* ```Transported```: **variable de salida**, indica si el pasajero fue 
  transportado a otra dimensión o no.

# Recodificación de variables
Para comenzar, se recodifican algunas de las variables expuestas y se crean otras
que puede resultar de importancia.
## Recodificación de la variable ```cabin``` 
La variable ```Cabin```, con el formato *deck/num/side* podría ser dividida en 
tres variables diferentes: ```Cabin_deck```, ```Cabin_num``` y ```Cabin_side```:

In [ ]:
cabin_splitted = str_split(df$Cabin, '/', simplify = TRUE)
# id_splitted = str_split(df$PassengerId, '_', simplify = TRUE)

In [ ]:
colnames(id_splitted)=c('group_id','personal_id')
# head(id_splitted)

In [ ]:
table(cabin_splitted[, 1])

In [ ]:
table(cabin_splitted[, 3])

In [ ]:
df = df %>% mutate(Cabin_deck = cabin_splitted[, 1], 
                   Cabin_num = as.integer(cabin_splitted[, 2]), 
                   Cabin_side = cabin_splitted[, 3]) %>% select(-Cabin)
head(df)

Es apreciable que algunos de los caracteres son vacíos, estos son valores 
perdidos que serán tratados más adelante. 

## Creación de la variable ```Alone```
También, la variable ```PassengerId``` en el formato *gggg_pp* nos indica si los 
pasajeros viajaban solos o en grupo, creamos una variable que indique esto:

In [ ]:
df = df %>% mutate(Group = (str_split(PassengerId, '_', simplify = TRUE))[,1])
vector_group = df %>% count(Group) %>% filter(n > 1)
vector_group = vector_group$Group
df = df %>% mutate(Alone = ifelse(Group %in% vector_group, "False", "True"))

In [ ]:
head(df)

# Valores perdidos o *Missing values*
Una vez recodificadas las variables para entender mejor los datos, se procede a 
estudiar los valores perdidos del dataset ```train.csv```.

In [ ]:
missmap(df)

In [ ]:
print(table(df$HomePlanet))
print(table(df$Destination))
print(table(df$CryoSleep))
print(table(df$VIP))

Como se pudo observar con la variable ```Cabin```, los valores perdidos parecen 
estar registrados como caracteres vacíos, los convertimos en ```NA```:

In [ ]:
df$Cabin_deck[df$Cabin_deck == ""] = NA
df$Cabin_num[df$Cabin_num == ""] = NA
df$Cabin_side[df$Cabin_side == ""] = NA
df$HomePlanet[df$HomePlanet == ""] = NA
df$Destination[df$Destination == ""] = NA
df$CryoSleep[df$CryoSleep == ""] = NA
df$VIP[df$VIP == ""] = NA

In [ ]:
missmap(df)

En total, los valores perdidos no suponen un gran porcentaje del dataset 
completo, por lo que se evalúa cuantas instancias eliminaríamos de borrar 
aquellas con algún valor perdido:

In [ ]:
# Porcentaje de filas incompletas
sum(!complete.cases(df))/nrow(df)

De eliminar estas filas, eliminaríamos el $22\%$ de los datos, por lo que 
se descarta esta opción. Una imputación es necesaria.

## Missing values categóricos
Se comienza con aquellas variables categóricas. Como método de imputación 
general, se le asignará a cada ```NA``` la **moda** de su atributo:

In [ ]:
mode_hp = (df %>% group_by(HomePlanet) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$HomePlanet
mode_cabin_deck = (df %>% group_by(Cabin_deck) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$Cabin_deck
mode_cabin_side = (df %>% group_by(Cabin_side) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$Cabin_side
mode_dest = (df %>% group_by(Destination) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$Destination
mode_cs = (df %>% group_by(CryoSleep) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$CryoSleep
mode_vip = (df %>% group_by(VIP) %>% summarize(n = n()) %>% na.omit %>% top_n(1))$VIP


In [ ]:
df_mode = data.frame(mode_hp = mode_hp, mode_cabin_deck = mode_cabin_deck, mode_cabin_side = mode_cabin_side, mode_cs = mode_cs, mode_vip = mode_vip, mode_dest = mode_dest)
df_mode
write.csv(df_mode,'data/modes.csv')

In [ ]:
df_imputed = data.frame(df)

In [ ]:
df_imputed$Cabin_deck[is.na(df_imputed$Cabin_deck)] = mode_cabin_deck
df_imputed$Cabin_side[is.na(df_imputed$Cabin_side)] = mode_cabin_side
df_imputed$HomePlanet[is.na(df_imputed$HomePlanet)] = mode_hp
df_imputed$Destination[is.na(df_imputed$Destination)] = mode_dest
df_imputed$VIP[is.na(df_imputed$VIP)] = mode_vip

Sin embargo, llama la atención el atributo ```CryoSleep```. Se observa que, en 
los casos donde sus variables numéricas (correspondientes a los gastos) son 
nulas, hay valores perdidos también:

In [ ]:
df_imputed %>% filter(RoomService==0,FoodCourt == 0, ShoppingMall == 0, Spa == 0,
                     VRDeck == 0) %>% count(CryoSleep)

En concreto hay 87 valores perdidos, 2690 casos donde esto se cumple y 470 donde 
no. Dado que la mayoría de pasajeros que no han gastado dinero en el viaje son 
personas en animación suspendida, se imputan estos 87 valores perdidos como 
```CryoSleep = 'True'```.

In [ ]:
cs_idTrue = df_imputed %>% filter(is.na(CryoSleep),RoomService==0,
                                  FoodCourt == 0, ShoppingMall == 0, Spa == 0,
                                  VRDeck == 0) %>% select(PassengerId)

df_imputed[df_imputed$PassengerId %in% cs_idTrue[,1],]$CryoSleep = 'True'

Por el contrario, si alguno de los gastos es distinto de cero, se podría 
considerar que el pasajero no está en animación suspendida:

In [ ]:
df_imputed %>% filter(RoomService!=0 | FoodCourt != 0 | ShoppingMall != 0 | Spa != 0 |
                     VRDeck != 0) %>% count(CryoSleep)

En efecto, todas las personas que han tenido algún gasto en la nave no están en 
animación suspendida a excepción de los valores perdidos. Los imputamos como 
```CryoSleep = 'False'```:

In [ ]:
cs_idFalse = df_imputed %>% filter(is.na(CryoSleep), (RoomService!=0 | FoodCourt != 0 | ShoppingMall != 0 | Spa != 0 |
                     VRDeck != 0)) %>% select(PassengerId) 

In [ ]:
df_imputed[df_imputed$PassengerId %in% cs_idFalse[,1],]$CryoSleep = 'False'

In [ ]:
df_imputed %>% filter(is.na(CryoSleep))

Pero sigue habiendo 11 valores perdidos en esta variable en los casos donde 
alguno de los gastos también son ```NA```. Se comprueba que son 11 casos:

In [ ]:
df_imputed %>% filter((is.na(RoomService) | RoomService == 0),
                      (is.na(FoodCourt) | FoodCourt == 0), 
                      (is.na(Spa) | Spa == 0), 
                      (is.na(ShoppingMall) | ShoppingMall == 0), 
                      (is.na(VRDeck) | VRDeck == 0)) %>%
               mutate(total_expenses = RoomService + FoodCourt + Spa + ShoppingMall + VRDeck) %>%
               filter(is.na(total_expenses)) %>% count(CryoSleep)

Imputamos estos casos como ```CryoSleep = 'True'```, dado que es extraño que 
los pasajeros solo gasten en uno de los servicios y que justo ese sea un valor 
perdido también:

In [ ]:
cs_idTrue_NAnum = df_imputed %>% filter(is.na(CryoSleep)) %>% select(PassengerId)

In [ ]:
df_imputed[df_imputed$PassengerId %in% cs_idTrue_NAnum[,1],]$CryoSleep = 'True'

In [ ]:
df_imputed %>% count(CryoSleep)

## Missing values numéricos
Procedemos ahora con los valores perdidos numéricos.

In [ ]:
missmap(df_imputed)

Comenzamos con los casos donde alguno de los gastos es un valor perdido, el 
resto son cero y están en animación suspendida:

In [ ]:
df_imputed %>% filter(CryoSleep == "True", (is.na(RoomService) | is.na(VRDeck) | is.na(Spa) | is.na(ShoppingMall) | is.na(FoodCourt))) %>%
                  mutate(total_expenses = RoomService + FoodCourt + Spa + ShoppingMall + VRDeck) %>%
                  filter(is.na(total_expenses)) %>% head()

Como mencionamos anteriormente, las personas en animación suspendida no han 
podido tener gastos, por lo que estos valores perdidos deben ser cero:

In [ ]:
expenses_IdZero_RoomService = df_imputed %>% filter(CryoSleep == "True", is.na(RoomService)) %>% select(PassengerId)
expenses_IdZero_VRDeck = df_imputed %>% filter(CryoSleep == "True", is.na(VRDeck)) %>% select(PassengerId)
expenses_IdZero_Spa = df_imputed %>% filter(CryoSleep == "True", is.na(Spa)) %>% select(PassengerId)
expenses_IdZero_ShoppingMall = df_imputed %>% filter(CryoSleep == "True", is.na(ShoppingMall)) %>% select(PassengerId)
expenses_IdZero_FoodCourt = df_imputed %>% filter(CryoSleep == "True", is.na(FoodCourt)) %>% select(PassengerId)

In [ ]:
df_imputed[df_imputed$PassengerId %in% expenses_IdZero_RoomService[,1],]$RoomService = 0
df_imputed[df_imputed$PassengerId %in% expenses_IdZero_VRDeck[,1],]$VRDeck = 0
df_imputed[df_imputed$PassengerId %in% expenses_IdZero_Spa[,1],]$Spa = 0
df_imputed[df_imputed$PassengerId %in% expenses_IdZero_ShoppingMall[,1],]$ShoppingMall = 0
df_imputed[df_imputed$PassengerId %in% expenses_IdZero_FoodCourt[,1],]$FoodCourt = 0

In [ ]:
missmap(df_imputed)

Comprobamos los casos donde ```CryoSleep = 'False'``` y hay algún valor 
perdido en los gastos:

In [ ]:
df_imputed %>% filter(CryoSleep == "False", (is.na(RoomService) | is.na(VRDeck) |
                      is.na(Spa) | is.na(ShoppingMall) | is.na(FoodCourt))) %>% 
                      head()

En estas observaciones, dada las distribuciones de las variables, se imputan 
los ```NA``` por las medianas de sus atributos:

In [ ]:
expenses_IdMedian_RoomService = df_imputed %>% filter(CryoSleep == "False", is.na(RoomService)) %>% select(PassengerId)
expenses_IdMedian_VRDeck = df_imputed %>% filter(CryoSleep == "False", is.na(VRDeck)) %>% select(PassengerId)
expenses_IdMedian_Spa = df_imputed %>% filter(CryoSleep == "False", is.na(Spa)) %>% select(PassengerId)
expenses_IdMedian_ShoppingMall = df_imputed %>% filter(CryoSleep == "False", is.na(ShoppingMall)) %>% select(PassengerId)
expenses_IdMedian_FoodCourt = df_imputed %>% filter(CryoSleep == "False", is.na(FoodCourt)) %>% select(PassengerId)

In [ ]:
df_imputed[df_imputed$PassengerId %in% expenses_IdMedian_RoomService[,1],]$RoomService = median(df_imputed$RoomService, na.rm = TRUE)
df_imputed[df_imputed$PassengerId %in% expenses_IdMedian_VRDeck[,1],]$VRDeck = median(df_imputed$VRDeck, na.rm = TRUE)
df_imputed[df_imputed$PassengerId %in% expenses_IdMedian_Spa[,1],]$Spa = median(df_imputed$Spa, na.rm = TRUE)
df_imputed[df_imputed$PassengerId %in% expenses_IdMedian_ShoppingMall[,1],]$ShoppingMall = median(df_imputed$ShoppingMall, na.rm = TRUE)
df_imputed[df_imputed$PassengerId %in% expenses_IdMedian_FoodCourt[,1],]$FoodCourt = median(df_imputed$FoodCourt, na.rm = TRUE)

In [ ]:
missmap(df_imputed)

In [ ]:
df_imputed[is.na(df_imputed$Age), ]$Age = median(df_imputed$Age, na.rm = TRUE)

In [ ]:
missmap(df_imputed)

In [ ]:
df_imputed %>% filter(CryoSleep == "True") %>% count(Cabin_side, Cabin_deck)

In [ ]:
df_imputed %>% filter(VIP == "True") %>% count(CryoSleep)

In [ ]:
df_imputed %>% filter(VIP == "True") %>% count(HomePlanet)

In [ ]:
df_imputed %>% count(Cabin_side, HomePlanet, Cabin_deck)

In [ ]:
df_imputed %>% count(HomePlanet, VIP)

### Variable solo o acompañado

In [ ]:
df_imputed = df_imputed %>% mutate(Group = (str_split(PassengerId, '_', simplify = TRUE))[,1])
vector_group = df_imputed %>% count(Group) %>% filter(n > 1)
vector_group = vector_group$Group
df_imputed = df_imputed %>% mutate(Alone = ifelse(Group %in% vector_group, "False", "True"))

In [ ]:
df_imputed

Los que están solos es más probable que hayan sido transportados, o al revés?

In [ ]:
df_imputed %>% count(Transported, Alone)

### Distribuciones variables numéricas

In [ ]:
ggplot(df_imputed, aes(x = Age, fill = Transported)) +
    geom_density() +
    facet_wrap(~Transported)

In [ ]:
str(df_imputed)

In [ ]:
hist_plot = function(variab,bins_=NULL,binwidth_=NULL){
    ggplot(df_imputed,aes(df_imputed[,variab],fill=Transported)) +
    geom_histogram(bins=bins_,binwidth=binwidth_) +
    labs(x = colnames(df_imputed)[variab])+ facet_wrap(~CryoSleep)
}


box_plot = function(variab){
    ggplot(df_imputed,aes(y=df_imputed[,variab],fill=Transported)) + geom_boxplot()+
    labs(y = colnames(df_imputed)[variab])+
    facet_wrap(~CryoSleep)
}

In [ ]:
hist_plot(5)
box_plot(5)

In [ ]:
hist_plot(8)
box_plot(8)

In [ ]:
hist_plot(9)
box_plot(9)

In [ ]:
hist_plot(10)
box_plot(10)

In [ ]:
hist_plot(11)
box_plot(11)

### Análisis univariable

In [ ]:
ggplot(df_imputed, aes(x = Destination)) +
    geom_bar() +
    facet_wrap(~Transported)

In [ ]:
ggplot(df_imputed, aes(x = HomePlanet)) +
    geom_bar() +
    facet_wrap(~Transported)

In [ ]:
ggplot(df, aes(x = HomePlanet)) +
    geom_bar() +
    facet_wrap(~Transported)

In [ ]:
# Ricardo: parece que es más probable de ser transportado si un individuo es de otro planeta que no sea la Tierra,  
# aunque esta sea la categoría con mayor número de individuos, por lo que el planeta de procedencia puede ser una primera
# pista para la clasificación.
ggplot(df, aes(x = HomePlanet, fill=Transported)) +
    geom_bar(color='black', alpha=0.5, position='identity') +
    scale_y_continuous(breaks = seq(0,3000,100))


In [ ]:
ggplot(df, aes(x = VIP, fill=Transported)) +
    geom_bar(color='black', alpha=0.5, position='identity') +
    scale_y_continuous(breaks = seq(0,5000,200))

In [ ]:
# Ricardo: ¿Por qué en el rango entre 800 y 1100 hay más transportados?

ggplot(df %>% filter(CryoSleep == "True"), aes(x = Cabin_num,fill=Transported)) +
    geom_density(alpha=0.5)+
scale_x_continuous(breaks = seq(0,1800,200))

In [ ]:
ggplot(df, aes(x = Cabin_num,fill=Transported)) +
    geom_bar()+
scale_x_continuous(breaks = seq(0,1800,200))

In [ ]:
str(df)

### ANÁLISIS BIVARIABLE


In [ ]:
ggplot(df_imputed,aes(x = RoomService,fill=Transported))+
        geom_density()

In [ ]:
table(df$CryoSleep)

In [ ]:
set_plot_dimensions(14,8)

ggplot(df,aes(x = RoomService,fill=CryoSleep))+
        geom_density() + facet_wrap(~CryoSleep,scales='free')


ggplot(df,aes(x = RoomService,fill=CryoSleep))+
        geom_boxplot()

In [ ]:
df1 = df_imputed %>% filter(CryoSleep=='True') %>% select_if(is.numeric) %>% 
     select(-Cabin_num)

head(df1)
table(df1$RoomService)

In [ ]:
df %>% filter(CryoSleep=='True',VIP=='True')

In [ ]:
df = df_imputed %>% mutate(total_expens = RoomService + FoodCourt + ShoppingMall + Spa +
                  VRDeck)

In [ ]:
df_imputed %>% group_by(Transported) %>%summarize(mean_expens = mean(total_expens,na.rm=TRUE)) 

In [ ]:
df_imputed %>% group_by(VIP) %>%summarize(mean_expens = mean(total_expens,na.rm=TRUE))

In [ ]:

df_imputed %>% group_by(CryoSleep) %>%summarize(mean_expens = mean(total_expens,na.rm=TRUE))

## Missing values en el conjunto de test

In [ ]:
# Cargamos datos
df_test = read.csv("data/test.csv", header = TRUE)
head(df_test)

In [ ]:
# Lo mismo que antes, dividimos la variable Cabin en sus componentes
splitted_test = str_split(df_test$Cabin, '/', simplify = TRUE)

In [ ]:
df_test = df_test %>% mutate(Cabin_deck = splitted_test[, 1], 
                   Cabin_num = as.integer(splitted_test[, 2]), 
                   Cabin_side = splitted_test[, 3]) %>% select(-Cabin)
head(df_test)

In [ ]:
df_test$Cabin_deck[df_test$Cabin_deck == ""] = NA
df_test$Cabin_num[df_test$Cabin_num == ""] = NA
df_test$Cabin_side[df_test$Cabin_side == ""] = NA
df_test$HomePlanet[df_test$HomePlanet == ""] = NA
df_test$Destination[df_test$Destination == ""] = NA
df_test$CryoSleep[df_test$CryoSleep == ""] = NA
df_test$VIP[df_test$VIP == ""] = NA

In [ ]:
missmap(df_test)
# Ricardo: efectivamente hay valores perdidos en el conjunto de test

In [ ]:
# Porcentaje de filas completas para el conjunto de test
sum(complete.cases(df_test))/nrow(df_test)

# Ricardo: la proporción de datos completos es muy parecida al conjunto de entrenamiento.

In [ ]:
# Comentarios generales:

# Ricardo (28/12/2022):
# -Os he dejado algunos comentarios en los que he puesto mi nombre.
# -Creo que deberíamos dejar bonito el notebook para entregarlo, para ello igual nos conviene poner un apartado definiendo
# el problema y describiendo brevemente las variables, además de ir incluyendo los pasos que hemos seguido en Introducción a la 
# ciencia de datos como apartados. Igual no hacer un análisis tan exhaustivo, pero lo suficiente para que conozcamos 
# bien las variables. Aunque igual podemos utilizar este notebook como borrador.
# -Por otro lado propongo dejar esta celda al final del documento por si queréis ir añadiendo comentarios para el resto,
# incluyendo vuestro nombre y añadiendo la fecha para dejar como una especie de "registro" y así
# no saturamos el grupo de whats app